In [5]:
!pip install tqdm
!pip install tqdm --upgrade

Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.47.0)


## Librerías

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np

# TESORFLOW
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Dense, Activation, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, SGD
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from IPython.display import Image

from tqdm.keras import TqdmCallback

In [6]:
print(tf.__version__)

2.2.0


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
#!unzip '/content/drive/My Drive/Máster KSchool/TFM/data/filtered_img.zip'

## Importar datos

In [9]:
data_tab = pd.read_csv('/content/drive/My Drive/Máster KSchool/TFM/data/filtered_df_with_imgNames.csv',sep=',',header= 0,usecols=['CODE','RATING'])
data_tab = data_tab.apply(lambda x: x.str.replace(',','.'))

In [10]:
data_tab.head(3)

,CODE,RATING
0,tt4686108.png,5.3
1,tt0445033.png,6.4
2,tt1167599.png,5.5


Categorizamos la columna Rating


In [11]:
data_tab['RATING']  = data_tab['RATING'].astype(float)
data_tab['RATING'] = pd.cut( data_tab['RATING'], bins = [0,5, 6.5,10], labels=['1','2','3'])
data_tab.head(3)

,CODE,RATING
0,tt4686108.png,2
1,tt0445033.png,2
2,tt1167599.png,2


In [12]:
print(len(data_tab[data_tab['RATING']=='2']))
print(len(data_tab[data_tab['RATING']=='1']))
print(len(data_tab[data_tab['RATING']=='0']))

47141
19770
0


Image data generator

In [13]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
                        dataframe=data_tab,
                        directory="filtered_img/",
                        x_col="CODE",
                        y_col="RATING",
                        subset="training",
                        batch_size=300,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(268, 182))
valid_generator=datagen.flow_from_dataframe(
                        dataframe=data_tab,
                        directory="filtered_img/",
                        x_col="CODE",
                        y_col="RATING",
                        subset="validation",
                        batch_size=300,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(268,182))

Found 80386 validated image filenames belonging to 3 classes.
Found 26795 validated image filenames belonging to 3 classes.


## Paramentros para el entrenamiento

In [14]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

epochs = 100

## Creamos y entrenamos el modelo

In [21]:
plot_model(base_model, to_file='model3.png',show_shapes=True,show_layer_names=True)
Image(retina=True, filename='model3.png')

Output hidden; open in https://colab.research.google.com to view.

In [16]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 268, 182, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 133, 90, 32)  864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 133, 90, 32)  96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 133, 90, 32)  0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/My Drive/Máster KSchool/TFM/Codigos/Img classification/IMG_Clasiffier-XCeption-Best_model_weights.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='min', period=1)

# Creamos el modelo base sobre el que vamos a hacer fine tunning
input_tensor = tf.keras.layers.Input((268,182,3))
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

# Creamos el clasificador
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# first: Entrenamos solo las capas del clasificador y congelamos las demas capas del modelo pretrained
for layer in base_model.layers:
    layer.trainable = False

# compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])

# train the model on the new data for a few epochs
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=epochs,callbacks=[TqdmCallback()], verbose=2)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=["accuracy"])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=epochs,callbacks=[TqdmCallback(),checkpoint], verbose=2)

Epoch 1/100
267/267 - 197s - loss: 1.0603 - accuracy: 0.4361 - val_loss: 1.0249 - val_accuracy: 0.4452
Epoch 2/100
267/267 - 229s - loss: 1.0158 - accuracy: 0.4596 - val_loss: 1.0067 - val_accuracy: 0.4733
Epoch 3/100
267/267 - 193s - loss: 1.0058 - accuracy: 0.4715 - val_loss: 0.9996 - val_accuracy: 0.4740
Epoch 4/100
267/267 - 193s - loss: 1.0022 - accuracy: 0.4758 - val_loss: 1.0012 - val_accuracy: 0.4683
Epoch 5/100
267/267 - 194s - loss: 0.9965 - accuracy: 0.4801 - val_loss: 0.9972 - val_accuracy: 0.4810
Epoch 6/100
267/267 - 194s - loss: 0.9911 - accuracy: 0.4852 - val_loss: 0.9962 - val_accuracy: 0.4774
Epoch 7/100
267/267 - 193s - loss: 0.9866 - accuracy: 0.4872 - val_loss: 1.0121 - val_accuracy: 0.4689
Epoch 8/100
267/267 - 194s - loss: 0.9792 - accuracy: 0.4934 - val_loss: 1.0111 - val_accuracy: 0.4655
Epoch 9/100
267/267 - 193s - loss: 0.9734 - accuracy: 0.4980 - val_loss: 0.9963 - val_accuracy: 0.4743
Epoch 10/100
267/267 - 194s - loss: 0.9670 - accuracy: 0.5018 - val_loss:

In [ ]:
print(history.history.keys())
#  "Accuracy"
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
!nvidia-smi